In [1]:
from glob import glob
from os import path as osp
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import pointbiserialr  # Point-biserial correlation coefficient

import sys
from os import pardir, getcwd
sys.path.append(osp.abspath(osp.join(getcwd(), pardir, pardir)))

from configuration import Config
config = Config()

In [2]:
def parse_filename(filename):
    filename = osp.split(filename)[-1]
    filename_root = osp.splitext(filename)[0]
    
    dataset, split, architecture, context_type, noise = filename_root.split('_', 4)
    noise = float(noise.replace('noise:', '').replace('-', '.'))
    
    return dataset, split, architecture, context_type, noise

In [3]:
# load annotations
ann_dir = osp.join(config.project_path, 'generated/compiled_annotations')
fig_location = osp.join(config.project_path, 'generated/figures')

noise_setting = '0-0'

nocontext_path = osp.join(ann_dir, f'refcoco_testb_trf_nocontext_noise:{noise_setting}.csv')
global_path = osp.join(ann_dir, f'refcoco_testb_trf_global_noise:{noise_setting}.csv')
scene_path = osp.join(ann_dir, f'refcoco_testb_trf_scene_noise:{noise_setting}.csv')
nocontext_cc_path = osp.join(ann_dir, f'refcoco_testb_clpgpt_nocontext_noise:{noise_setting}.csv')
global_cc_path = osp.join(ann_dir, f'refcoco_testb_clpgpt_global_noise:{noise_setting}.csv')
scene_cc_path = osp.join(ann_dir, f'refcoco_testb_clpgpt_scene_noise:{noise_setting}.csv')

nocontext_df = pd.read_csv(nocontext_path)
global_df = pd.read_csv(global_path)
scene_df = pd.read_csv(scene_path)
global_cc_df = pd.read_csv(global_cc_path)
nocontext_cc_df = pd.read_csv(nocontext_cc_path)
scene_cc_df = pd.read_csv(scene_cc_path)

# load target class coverage
target_class_ratio_path = osp.join(config.project_path, 'generated/target_context_classes/refcoco-testB-target-class-in-context.json')
target_class_ratio = pd.read_json(target_class_ratio_path).set_index('ann_id').target_class_in_context_ratio

In [4]:
# merge to combined DataFrame

merged_df = pd.merge(
    global_df[['idx', 'ann_id', 'expression', 'majority_annotation']].rename(
        columns={'majority_annotation':'global_annotation'}),
    scene_df[['ann_id', 'majority_annotation']].rename(
        columns={'majority_annotation':'scene_annotation'}),
    left_on='ann_id', right_on='ann_id'
)

merged_df = pd.merge(
    merged_df, 
    nocontext_df[['ann_id', 'majority_annotation']].rename(
        columns={'majority_annotation':'nocontext_annotation'}),
)

merged_df = pd.merge(
    merged_df, 
    global_cc_df[['ann_id', 'majority_annotation']].rename(
        columns={'majority_annotation':'global_cc_annotation'}),
)

merged_df = pd.merge(
    merged_df, 
    nocontext_cc_df[['ann_id', 'majority_annotation']].rename(
        columns={'majority_annotation':'nocontext_cc_annotation'}),
)

merged_df = pd.merge(
    merged_df, 
    scene_cc_df[['ann_id', 'majority_annotation']].rename(
        columns={'majority_annotation':'scene_cc_annotation'}),
)


merged_df = merged_df.set_index('ann_id')
merged_df['target_class_context_ratio'] = target_class_ratio.loc[merged_df.index]

In [5]:
merged_df.head()

,idx,expression,global_annotation,scene_annotation,nocontext_annotation,global_cc_annotation,nocontext_cc_annotation,scene_cc_annotation,target_class_context_ratio
ann_id,,,,,,,,,
100002,0,couch on right,A,A,A,A,A,A,0.224132
120013,1,table top right,A,A,A,A,F,F,0.212829
573794,2,bench on left,A,A,A,A,F,M,0.049891
586064,3,right bear,A,A,A,A,A,A,0.131438
314750,4,bed,A,A,A,A,F,F,0.002457


In [6]:
# global / scene model hits (binary format)

def is_hit(value):
    return value == 'A'

merged_df['nocontext_hit'] = merged_df.nocontext_annotation.map(is_hit)
merged_df['global_hit'] = merged_df.global_annotation.map(is_hit)
merged_df['scene_hit'] = merged_df.scene_annotation.map(is_hit)
merged_df['nocontext_cc_hit'] = merged_df.nocontext_cc_annotation.map(is_hit)
merged_df['global_cc_hit'] = merged_df.global_cc_annotation.map(is_hit)
merged_df['scene_cc_hit'] = merged_df.scene_cc_annotation.map(is_hit)

# combined stats
both_hits = np.logical_and(merged_df.global_hit, merged_df.scene_hit)
#none_hits = ~np.logical_or(merged_df.global_hit, merged_df.scene_hit)
none_hits = merged_df.apply(lambda x: ~np.logical_or.reduce((x.global_hit, x.scene_hit, x.global_cc_hit)), axis=1)

merged_df['both_hit'] = both_hits
merged_df['none_hit'] = none_hits

merged_df.head()

,idx,expression,global_annotation,scene_annotation,nocontext_annotation,global_cc_annotation,nocontext_cc_annotation,scene_cc_annotation,target_class_context_ratio,nocontext_hit,global_hit,scene_hit,nocontext_cc_hit,global_cc_hit,scene_cc_hit,both_hit,none_hit
ann_id,,,,,,,,,,,,,,,,,
100002,0,couch on right,A,A,A,A,A,A,0.224132,True,True,True,True,True,True,True,False
120013,1,table top right,A,A,A,A,F,F,0.212829,True,True,True,False,True,False,True,False
573794,2,bench on left,A,A,A,A,F,M,0.049891,True,True,True,False,True,False,True,False
586064,3,right bear,A,A,A,A,A,A,0.131438,True,True,True,True,True,True,True,False
314750,4,bed,A,A,A,A,F,F,0.002457,True,True,True,False,True,False,True,False


In [7]:
print('nocontext hit vs. target class ratio')
global_result = pointbiserialr(merged_df.nocontext_hit, merged_df.target_class_context_ratio)  # correlated
print(f"""
correlation: {round(global_result.correlation, 3)}
p value: {global_result.pvalue} 
p < 0.05: {global_result.pvalue < 0.05}
p < 0.01: {global_result.pvalue < 0.01}
p < 0.001: {global_result.pvalue < 0.001}
""".strip()
)

mean_tcr = merged_df[merged_df.nocontext_hit].target_class_context_ratio.mean()
print(f'mean target class ratio for this subset: {round(mean_tcr, 2)}')

nocontext hit vs. target class ratio
correlation: 0.128
p value: 0.0699194903707792 
p < 0.05: False
p < 0.01: False
p < 0.001: False
mean target class ratio for this subset: 0.19


In [8]:
print('global hit vs. target class ratio')
global_result = pointbiserialr(merged_df.global_hit, merged_df.target_class_context_ratio)  # correlated
print(f"""
correlation: {round(global_result.correlation, 3)}
p value: {global_result.pvalue} 
p < 0.05: {global_result.pvalue < 0.05}
p < 0.01: {global_result.pvalue < 0.01}
p < 0.001: {global_result.pvalue < 0.001}
""".strip()
)

mean_tcr = merged_df[merged_df.global_hit].target_class_context_ratio.mean()
print(f'mean target class ratio for this subset: {round(mean_tcr, 2)}')

global hit vs. target class ratio
correlation: 0.109
p value: 0.12585856589327324 
p < 0.05: False
p < 0.01: False
p < 0.001: False
mean target class ratio for this subset: 0.19


In [9]:
print('scene hit vs. target class ratio')
scene_result = pointbiserialr(merged_df.scene_hit, merged_df.target_class_context_ratio)  # correlated
print(f"""
correlation: {round(scene_result.correlation, 3)}
p value: {scene_result.pvalue} 
p < 0.05: {scene_result.pvalue < 0.05}
p < 0.01: {scene_result.pvalue < 0.01}
p < 0.001: {scene_result.pvalue < 0.001}
""".strip()
)


mean_tcr = merged_df[merged_df.global_hit].target_class_context_ratio.mean()
print(f'mean target class ratio for this subset: {round(mean_tcr, 2)}')

scene hit vs. target class ratio
correlation: 0.154
p value: 0.0298416182029029 
p < 0.05: True
p < 0.01: False
p < 0.001: False
mean target class ratio for this subset: 0.19


In [10]:
print('nocontext cc hit vs. target class ratio')
global_result = pointbiserialr(merged_df.nocontext_cc_hit, merged_df.target_class_context_ratio)  # correlated
print(f"""
correlation: {round(global_result.correlation, 3)}
p value: {global_result.pvalue} 
p < 0.05: {global_result.pvalue < 0.05}
p < 0.01: {global_result.pvalue < 0.01}
p < 0.001: {global_result.pvalue < 0.001}
""".strip()
)

mean_tcr = merged_df[merged_df.nocontext_cc_hit].target_class_context_ratio.mean()
print(f'mean target class ratio for this subset: {round(mean_tcr, 2)}')

nocontext cc hit vs. target class ratio
correlation: 0.156
p value: 0.02743901824216711 
p < 0.05: True
p < 0.01: False
p < 0.001: False
mean target class ratio for this subset: 0.21


In [11]:
print('global cc hit vs. target class ratio')
global_result = pointbiserialr(merged_df.global_cc_hit, merged_df.target_class_context_ratio)  # correlated
print(f"""
correlation: {round(global_result.correlation, 3)}
p value: {global_result.pvalue} 
p < 0.05: {global_result.pvalue < 0.05}
p < 0.01: {global_result.pvalue < 0.01}
p < 0.001: {global_result.pvalue < 0.001}
""".strip()
)

mean_tcr = merged_df[merged_df.global_cc_hit].target_class_context_ratio.mean()
print(f'mean target class ratio for this subset: {round(mean_tcr, 2)}')

global cc hit vs. target class ratio
correlation: 0.142
p value: 0.04448568889599227 
p < 0.05: True
p < 0.01: False
p < 0.001: False
mean target class ratio for this subset: 0.2


In [12]:
print('scene cc hit vs. target class ratio')
global_result = pointbiserialr(merged_df.scene_cc_hit, merged_df.target_class_context_ratio)  # correlated
print(f"""
correlation: {round(global_result.correlation, 3)}
p value: {global_result.pvalue} 
p < 0.05: {global_result.pvalue < 0.05}
p < 0.01: {global_result.pvalue < 0.01}
p < 0.001: {global_result.pvalue < 0.001}
""".strip()
)

mean_tcr = merged_df[merged_df.global_cc_hit].target_class_context_ratio.mean()
print(f'mean target class ratio for this subset: {round(mean_tcr, 2)}')

scene cc hit vs. target class ratio
correlation: 0.353
p value: 3.0107525120939705e-07 
p < 0.05: True
p < 0.01: True
p < 0.001: True
mean target class ratio for this subset: 0.2


In [13]:
print('none hit vs. target class ratio')
none_result = pointbiserialr(merged_df.none_hit, merged_df.target_class_context_ratio)  # correlated
print(f"""
correlation: {round(none_result.correlation, 3)}
p value: {none_result.pvalue} 
p < 0.05: {none_result.pvalue < 0.05}
p < 0.01: {none_result.pvalue < 0.01}
p < 0.001: {none_result.pvalue < 0.001}
""".strip()
)


mean_tcr = merged_df[merged_df.none_hit].target_class_context_ratio.mean()
print(f'mean target class ratio for this subset: {round(mean_tcr, 2)}')

none hit vs. target class ratio
correlation: -0.057
p value: 0.42035529519311104 
p < 0.05: False
p < 0.01: False
p < 0.001: False
mean target class ratio for this subset: 0.15
